<a href="https://colab.research.google.com/github/Angel-dash/Abstractive-Text-Summarization/blob/main/DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!unzip /content/drive/MyDrive/MiniProjectdata/data.zip


Archive:  /content/drive/MyDrive/MiniProjectdata/data.zip
  inflating: cnn_dailymail/test.csv  
  inflating: cnn_dailymail/train.csv  
  inflating: cnn_dailymail/validation.csv  


In [3]:
import numpy as np
import pandas as pd
import re
from bs4 import BeautifulSoup
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import warnings
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")
import textwrap
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
data_train=pd.read_csv('/content/cnn_dailymail/train.csv')
data_test=pd.read_csv('/content/cnn_dailymail/test.csv')
data_validation=pd.read_csv('/content/cnn_dailymail/validation.csv')

Data preprocessing

In [5]:
data = pd.concat([data_train, data_test, data_validation ])
df = data

In [6]:
new_df=df.sample(75000)

In [7]:
new_df.head(3)

,id,article,highlights
168774,6653f2262ad00034f880db697c16a903d41c977c,"Peter Kassig first went to the Middle East as a U.S. soldier and returned as a medical worker, feeling compelled to help victims of war. ""We each get one life and that's it. We get one shot at thi...","American Peter Kassig, 26, helped treat wounded Syrians .\nHe served in the U.S. military in Iraq but looked for a ""game changer"" after returning home .\nHis parents in Indianapolis, Indiana, con..."
1469,071d12740fe08c476a591ddffb4fb620e647f1da,"Winner: Simon Wood took home the TV crown . A single father of four was crowned MasterChef champion last night, fulfilling a dream to become a chef that he has nurtured since he was eight. Simon W...","Single father Simon Wood, 38, fulfilled childhood dream of becoming a chef .\nHe beat mother-of-four Emma Spitzer and Tony Rodd, 33, from London .\nThree finalists were challenged to cook a three-..."
613,07d4f94c79db22935db60a1b0e1b9f4b6f514949,"(CNN)ISIS militants in northern Syria released 19 Assyrian Christian hostages Sunday, an activist group said. According to the London-based Syrian Observatory for Human Rights, all but one of the ...","It's unclear what will become of the other kidnapped Assyrians .\nSyrian Observatory: ISIS has executed nearly 2,000 people since June ."


In [8]:
new_df.shape

(75000, 3)

In [9]:
sample=new_df.sample()
print("****Article****")
print(sample.article.values)
print('****Highlights****')
print(sample.highlights.values)

****Article****
['At the start of a big week for the Higgs boson, the most sought-after particle in all of physics, scientists in Illinois said Monday that they had crept closer to proving that the particle exists but had been unable to reach a definitive conclusion. The scientists outlined their final analysis based on more than 10 years of research and 500 trillion particle collisions using the U.S. Department of Energy\'s Fermilab Tevatron collider near Batavia, Illinois, whose budgetary woes shut it down last year. What is the Higgs boson and why is it important? Their announcement came two days before researchers at the Large Hadron Collider under the Alps are due to unveil their latest results at an eagerly awaited seminar at the CERN particle physics laboratory in Geneva, Switzerland. "Our data strongly point toward the existence of the Higgs boson," Rob Roser, a spokesman for one of two independent experiments at the Tevatron, said in a statement. "But it will take results from

Checking for Null values


In [10]:
new_df.isna().sum()

id            0
article       0
highlights    0
dtype: int64

Checking for duplicates


In [11]:
new_df.duplicated(subset=['article','highlights']).sum()

205

In [12]:
new_df=new_df.drop_duplicates(subset=['article','highlights'])
new_df.shape

(74795, 3)

In [13]:
new_df.duplicated(subset=['article','highlights']).sum()

0

In [14]:
new_df['article'][:10]

168774    Peter Kassig first went to the Middle East as a U.S. soldier and returned as a medical worker, feeling compelled to help victims of war. "We each get one life and that's it. We get one shot at thi...
1469      Winner: Simon Wood took home the TV crown . A single father of four was crowned MasterChef champion last night, fulfilling a dream to become a chef that he has nurtured since he was eight. Simon W...
613       (CNN)ISIS militants in northern Syria released 19 Assyrian Christian hostages Sunday, an activist group said. According to the London-based Syrian Observatory for Human Rights, all but one of the ...
121057    A two-year-old girl born without a windpipe has become the youngest patient in the world to receive a bio-engineered organ after doctors implanted her with a special tube grown from her own stem c...
158095    By . Michael Zennie . PUBLISHED: . 09:53 EST, 3 June 2013 . | . UPDATED: . 12:17 EST, 3 June 2013 . An entire family of seven - including four young c

In [15]:
new_df['article'] = new_df['article'].str.lower()
new_df['highlights'] = new_df['highlights'].str.lower()

new_df.head()

,id,article,highlights
168774,6653f2262ad00034f880db697c16a903d41c977c,"peter kassig first went to the middle east as a u.s. soldier and returned as a medical worker, feeling compelled to help victims of war. ""we each get one life and that's it. we get one shot at thi...","american peter kassig, 26, helped treat wounded syrians .\nhe served in the u.s. military in iraq but looked for a ""game changer"" after returning home .\nhis parents in indianapolis, indiana, con..."
1469,071d12740fe08c476a591ddffb4fb620e647f1da,"winner: simon wood took home the tv crown . a single father of four was crowned masterchef champion last night, fulfilling a dream to become a chef that he has nurtured since he was eight. simon w...","single father simon wood, 38, fulfilled childhood dream of becoming a chef .\nhe beat mother-of-four emma spitzer and tony rodd, 33, from london .\nthree finalists were challenged to cook a three-..."
613,07d4f94c79db22935db60a1b0e1b9f4b6f514949,"(cnn)isis militants in northern syria released 19 assyrian christian hostages sunday, an activist group said. according to the london-based syrian observatory for human rights, all but one of the ...","it's unclear what will become of the other kidnapped assyrians .\nsyrian observatory: isis has executed nearly 2,000 people since june ."
121057,287721dc0950d27e6260ebac9d3dfc04de4adb7d,a two-year-old girl born without a windpipe has become the youngest patient in the world to receive a bio-engineered organ after doctors implanted her with a special tube grown from her own stem c...,"hannah warren was born without a windpipe in 2010 and has been unable to breathe, eat, drink or swallow on her own since then .\ndoctors in her home country of south korea told her parents that sh..."
158095,586a1f0849b64c259294f98e21d60f05e95d98de,"by . michael zennie . published: . 09:53 est, 3 june 2013 . | . updated: . 12:17 est, 3 june 2013 . an entire family of seven - including four young children - was swept away and drowned by flood ...",samuel cifuentes led his wife florinda and son alex - along with his wife's cousin and her three children - to a drainage ditch near his house .\nall seven were killed in flash floods that swept t...


In [16]:
new_df['article']=new_df['article'].str.lower()

new_df['highlights']=new_df['highlights'].str.lower()

Converting to lower case

In [17]:
new_df.head()

,id,article,highlights
168774,6653f2262ad00034f880db697c16a903d41c977c,"peter kassig first went to the middle east as a u.s. soldier and returned as a medical worker, feeling compelled to help victims of war. ""we each get one life and that's it. we get one shot at thi...","american peter kassig, 26, helped treat wounded syrians .\nhe served in the u.s. military in iraq but looked for a ""game changer"" after returning home .\nhis parents in indianapolis, indiana, con..."
1469,071d12740fe08c476a591ddffb4fb620e647f1da,"winner: simon wood took home the tv crown . a single father of four was crowned masterchef champion last night, fulfilling a dream to become a chef that he has nurtured since he was eight. simon w...","single father simon wood, 38, fulfilled childhood dream of becoming a chef .\nhe beat mother-of-four emma spitzer and tony rodd, 33, from london .\nthree finalists were challenged to cook a three-..."
613,07d4f94c79db22935db60a1b0e1b9f4b6f514949,"(cnn)isis militants in northern syria released 19 assyrian christian hostages sunday, an activist group said. according to the london-based syrian observatory for human rights, all but one of the ...","it's unclear what will become of the other kidnapped assyrians .\nsyrian observatory: isis has executed nearly 2,000 people since june ."
121057,287721dc0950d27e6260ebac9d3dfc04de4adb7d,a two-year-old girl born without a windpipe has become the youngest patient in the world to receive a bio-engineered organ after doctors implanted her with a special tube grown from her own stem c...,"hannah warren was born without a windpipe in 2010 and has been unable to breathe, eat, drink or swallow on her own since then .\ndoctors in her home country of south korea told her parents that sh..."
158095,586a1f0849b64c259294f98e21d60f05e95d98de,"by . michael zennie . published: . 09:53 est, 3 june 2013 . | . updated: . 12:17 est, 3 june 2013 . an entire family of seven - including four young children - was swept away and drowned by flood ...",samuel cifuentes led his wife florinda and son alex - along with his wife's cousin and her three children - to a drainage ditch near his house .\nall seven were killed in flash floods that swept t...


In [18]:
#removing html tags
import re
def remove_html_tags(text):
    if isinstance(text, str):
        pattern = re.compile('<.*?>')
        return pattern.sub(r'', text)
    else:
        return text

In [19]:
new_df['article'] = new_df['article'].apply(remove_html_tags)

new_df['highlights'] = new_df['highlights'].apply(remove_html_tags)

In [20]:
new_df.head()

,id,article,highlights
168774,6653f2262ad00034f880db697c16a903d41c977c,"peter kassig first went to the middle east as a u.s. soldier and returned as a medical worker, feeling compelled to help victims of war. ""we each get one life and that's it. we get one shot at thi...","american peter kassig, 26, helped treat wounded syrians .\nhe served in the u.s. military in iraq but looked for a ""game changer"" after returning home .\nhis parents in indianapolis, indiana, con..."
1469,071d12740fe08c476a591ddffb4fb620e647f1da,"winner: simon wood took home the tv crown . a single father of four was crowned masterchef champion last night, fulfilling a dream to become a chef that he has nurtured since he was eight. simon w...","single father simon wood, 38, fulfilled childhood dream of becoming a chef .\nhe beat mother-of-four emma spitzer and tony rodd, 33, from london .\nthree finalists were challenged to cook a three-..."
613,07d4f94c79db22935db60a1b0e1b9f4b6f514949,"(cnn)isis militants in northern syria released 19 assyrian christian hostages sunday, an activist group said. according to the london-based syrian observatory for human rights, all but one of the ...","it's unclear what will become of the other kidnapped assyrians .\nsyrian observatory: isis has executed nearly 2,000 people since june ."
121057,287721dc0950d27e6260ebac9d3dfc04de4adb7d,a two-year-old girl born without a windpipe has become the youngest patient in the world to receive a bio-engineered organ after doctors implanted her with a special tube grown from her own stem c...,"hannah warren was born without a windpipe in 2010 and has been unable to breathe, eat, drink or swallow on her own since then .\ndoctors in her home country of south korea told her parents that sh..."
158095,586a1f0849b64c259294f98e21d60f05e95d98de,"by . michael zennie . published: . 09:53 est, 3 june 2013 . | . updated: . 12:17 est, 3 june 2013 . an entire family of seven - including four young children - was swept away and drowned by flood ...",samuel cifuentes led his wife florinda and son alex - along with his wife's cousin and her three children - to a drainage ditch near his house .\nall seven were killed in flash floods that swept t...


In [21]:
#removing punchuation


import string
string.punctuation
exclude = string.punctuation
def remove_punc(text):
    if isinstance(text, str):
        return ''.join(ch for ch in text if ch not in exclude)
    else:
        return str(text)  # Convert non-string data to string

In [22]:
new_df['article'] = new_df['article'].apply(remove_punc)

new_df['highlights'] = new_df['highlights'].apply(remove_punc)

In [23]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')

print(stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
new_df['article'] = new_df['article'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
new_df['highlights'] = new_df['highlights'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [25]:
data_train.head(1)

,id,article,highlights
0,0001d1afc246a7964130f43ae940af6bc6c57f01,"By . Associated Press . PUBLISHED: . 14:11 EST, 25 October 2013 . | . UPDATED: . 15:36 EST, 25 October 2013 . The bishop of the Fargo Catholic Diocese in North Dakota has exposed potentially hundr...","Bishop John Folda, of North Dakota, is taking time off after being diagnosed .\nHe contracted the infection through contaminated food in Italy .\nChurch members in Fargo, Grand Forks and Jamestown..."


In [26]:
new_df.head(1)

,id,article,highlights
168774,6653f2262ad00034f880db697c16a903d41c977c,peter kassig first went middle east us soldier returned medical worker feeling compelled help victims war get one life thats get one shot dont get doovers time put shut said 2012 interview cnn kas...,american peter kassig 26 helped treat wounded syrians served us military iraq looked game changer returning home parents indianapolis indiana confirmed hed taken hostage kassig converted islam hel...


In [27]:
#!pip install contractions


In [28]:
max_len_text=80
max_len_summary=10

In [29]:
from sklearn.model_selection import train_test_split
x_tr,x_val,y_tr,y_val=train_test_split(df['article'],df['highlights'],test_size=0.1,random_state=0,shuffle=True)

In [30]:
#prepare a tokenizer for reviews on training data
x_tokenizer = Tokenizer()
x_tokenizer.fit_on_texts(list(x_tr))

#convert text sequences into integer sequences
x_tr    =   x_tokenizer.texts_to_sequences(x_tr)
x_val   =   x_tokenizer.texts_to_sequences(x_val)

#padding zero upto maximum length
x_tr    =   pad_sequences(x_tr,  maxlen=max_len_text, padding='post')
x_val   =   pad_sequences(x_val, maxlen=max_len_text, padding='post')

x_voc_size   =  len(x_tokenizer.word_index) +1

In [31]:
#preparing a tokenizer for summary on training data
y_tokenizer = Tokenizer()
y_tokenizer.fit_on_texts(list(y_tr))

#convert summary sequences into integer sequences
y_tr    =   y_tokenizer.texts_to_sequences(y_tr)
y_val   =   y_tokenizer.texts_to_sequences(y_val)

#padding zero upto maximum length
y_tr    =   pad_sequences(y_tr, maxlen=max_len_summary, padding='post')
y_val   =   pad_sequences(y_val, maxlen=max_len_summary, padding='post')

y_voc_size  =   len(y_tokenizer.word_index) +1

LSTM model building

In [32]:
from keras import backend as K
K.clear_session()
latent_dim = 500

In [33]:
#encoder
encoder_inputs = Input(shape=(max_len_text,))
enc_emb = Embedding(x_voc_size, latent_dim,trainable=True)(encoder_inputs)

In [37]:
#LSTM 1
encoder_lstm1 = LSTM(latent_dim,return_sequences=True,return_state=True)
encoder_output1, state_h1, state_c1 = encoder_lstm1(enc_emb)

#LSTM2
encoder_lstm2 = LSTM(latent_dim,return_sequences=True,return_state=True)
encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1)

encoder_lstm3=LSTM(latent_dim, return_state=True, return_sequences=True)
encoder_outputs, state_h, state_c= encoder_lstm3(encoder_output2)

In [38]:
#decoder
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(y_voc_size, latent_dim,trainable=True)
dec_emb = dec_emb_layer(decoder_inputs)

In [39]:
#LSTM using encoder_states as initial state
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs,decoder_fwd_state, decoder_back_state = decoder_lstm(dec_emb,initial_state=[state_h, state_c])


In [40]:
#Attention Layer
Attention layer attn_layer = AttentionLayer(name='attention_layer')
attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs])

SyntaxError: ignored